## 1. 如何表示词语 ##

在NLP的所有任务中，第一个也是可以说是最重要的共同点是，我们如何表示词语并输入到其他的任意模型中。在早期NLP任务中，将单词视为原子符号。为了更好地完成大多数NLP任务，我们首先需要对单词之间的__相似性和差异性__有一些概念。使用**词向量**，我们可以很容易地在向量本身中编码这种能力(使用距离度量，如Jaccard、cos、Euclidean等)。

## 2. 词向量##
据估计，英语语言中1300万个单词记号(token)，但是这些单词之间是否完全没有关系?比如单词 "Feline"和"cat", "hotel"和"motel",很明显这些单词之间的确是存在某种关系的。因此，我们希望把每个单词编码为一个向量，它代表着词空间上的某个点。 这是众多原因中最重要的一点，但最直观的原因是也许确实存在某个n维空间(远小于1300万)足以编码我们的语言的所有的语义。每个维度都会编码一些我们使用语音传递的意义。例如，语义维度可能表示时态(过去、现在和将来)、计数(单数和复数)和性别(男性vs女性)。


## 3. one-hot编码 ##

那么让我们来研究第一个单词向量，也可以说是最简单的，一个ont-hot向量:每个单词可以用一一个大小为$R^{|V|\times 1}$的向量表示，该向量在除了该单词在词典的索引位置上是1外，其他位置都是0。在这个表示法中，$|V|$是词汇表的大小。这类编码中的单词向量将如下所示:
![one-hot](one-hot.png)

我们将每个单词表示为一个完全独立的实体。但是这样的单词表示没有直接给我们任何相似的概念,因为不同向量之间是正交的。也许我们可以试着把这个空间的大小从$R^{|V|}$缩小到更小的空间从而找到一个子空间来编码单词之间的关系。

## 4. 基于迭代的方法-Word2vec##

我们可以尝试创建一个模型，该模型能够一次学习一个迭代，并最终能够编码给定上下文的单词的概率，而不是计算和存储一些大型数据集(可能是数十亿个句子)的全局信息。其思想是设计一个参数为词向量的模型。然后，按照一定的目标训练模型。在每次迭代中，我们运行我们的模型，评估错误，并遵循一个更新规则，该规则具有惩罚导致错误的模型参数的概念。因此，我们学习单词向量。这很古老想法可以追溯到1986，我们将此方法称为“反向传播”错误。模型和任务越简单，训练它的速度就越快。
Word2vec是一个软件包，它实际上包括:
- 2个算法:continuous bag-of-words(CBOW)和skip-gram。
CBOW的目的是根据词的向量从周围的上下文中预测中心词。skip-gram则相反，它从中心词预测上下文词的分布(概率)。
- 2个训练方法:负采样和层次softmax。
负采样通过抽样负例子来定义一个目标，而层次softmax使用一个高效的树结构计算所有词汇表的概率来定义一个目标

## 5. CBOW ##

我们创建了两个矩阵，$V \in R^{n \times |V|}$和$U \in R^{|V|\times n}$。其中n是任意大小，它定义了嵌入空间的大小。
V为输入词矩阵，使得V的第i列为该模型的输入词$w_i$的n维嵌入向量。我们把这个n×1的向量表示为$v_i$。同样，U是输出词矩阵。U的第j行是词$w_j$的一个n维嵌入向量，它是模型的一个输出。我们把U的这一行记作$u_j$。注意，我们实际上学习了每个单词$w_i$的两个向量
(即输入字向量$v_i$和输出字向量$u_i$)。

训练步骤：
1. 我们为大小$m:(x^{c-m},...,x^{c-1}, x^{c+1}, ..., x^{c+m} \in R^{|V|})$的输入的上下文生成词向量的one-hot编码.
2. 得到了上下文的嵌入(embedded)词向量$(v_{c-m}=Vx^{c-m}, v_{c-m+1}=Vx^{c-m+1}, ..., v_{c+m}=Vx^{c+m}\in R)$
3. 求这些向量的平均 $\hat v = \frac{v_{c-m}+v_{c-m+1}+...+v_{c+m}}{2m}$
4. 计算得分向量 $z=U\hat v \in R^{|V|}$由于相似向量的点积越高，为了得到高分，它会把相似的单词聚集到一起。
5. 把分数转换为概率 $\hat y = softmax(x)\in R^{|V|}$
6. 我们希望自己生成概率,$\hat y \in R^{|V|}$,与真正的概率$y\in R^{|V|}$相匹配，也是实际的词的一个one-hot向量。

我们需要创建一个目标函数。通常，当我们试图从真实概率中学习概率时，我们会借助信息论来测量两个分布之间的距离。在这里,我们使用一个通用的的选择距离/损失的度量交叉熵 $H(\hat y, y) = -\sum_{j=1}^{|V|}y_j\log(\hat y_j)$.我们使用随机梯度下降更新所有相关的字向量$u_c$和$v_j$。
![cbow](cbow-loss-function.png)

## 6. Skip-Gram ##

比如对于句子"The cat _jump_ over the puddle"在Skip-Gram模型可以描述为：给定中心词"_jump_",该模型将能够预测或生成周围的单词“the”、“cat”、“over”、“the”、“puddle”。这里我们称"_jump_"为上下文。Skip-Gram的输入是CBOW的输出，而Skip-Gram的输出是CBOW的输入。


与CBOW一样，我们需要生成一个目标函数来评估模型。这里的一个关键区别是，我们调用了一个Naive贝叶斯假设来分解概率。这里使用的是一个强的(朴素的)条件独立假设。换句话说，给定中心字，所有输出字都是完全独立的。
![skip-gram-loss-function](skip-gram-loss-function.png)

注意到这个目标函数同样可以写成交叉熵的形式：
![skip-gram-loss-note](skip-gram-loss-note.png)

## 7. 负采样 ##

## 8. 层次softmax ##